In [ ]:
# 0. Colab 파일 업로드 (한 번만 필요)
from google.colab import files
print("필요 파일 업로드: metro JSON(data-metro-station-1.0.0.json,data-metro-line-1.0.0.json), bus Excel(bus_stops_seoul.xlsx,bus_route.xlsx) 파일을 올려주세요.")
uploaded = files.upload()

필요 파일 업로드: metro JSON(data-metro-station-1.0.0.json,data-metro-line-1.0.0.json), bus Excel(bus_stops_seoul.xlsx,bus_route.xlsx) 파일을 올려주세요.


Saving bus_route.xlsx to bus_route.xlsx
Saving bus_stops_seoul.xlsx to bus_stops_seoul.xlsx
Saving data-metro-line-1.0.0.json to data-metro-line-1.0.0.json
Saving data-metro-station-1.0.0.json to data-metro-station-1.0.0.json


In [ ]:
!pip install folium pandas networkx osmnx scipy geopy openpyxl cch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.5/112.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.2/100.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 338.0/338.0 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 67.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 115.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 27.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [ ]:
# 1. 라이브러리 설치 및 import
import json
import os
import pandas as pd
import networkx as nx
import osmnx as ox
import numpy as np
from osmnx.distance import nearest_nodes
from scipy.spatial import cKDTree
from geopy.geocoders import Nominatim
from itertools import combinations, islice
from math import radians, sin, cos, sqrt, asin
from multiprocessing import Pool
import cch
from geopy.geocoders import Nominatim
from concurrent.futures import ProcessPoolExecutor
import heapq
from itertools import product

In [ ]:
# ───────────────────────────────────────────────────────────────────────────────
# 2. 실제 도로망 전체 로드 (network_type='drive')
#    - 서울 전역을 대상으로 OSMnx의 drive 네트워크를 가져옵니다.
#    - 로그가 너무 많으면 아래 주석을 해제하고 설정을 바꿀 수 있습니다.
# ox.config(log_console=True, use_cache=True)

road_graph = ox.graph_from_place("Seoul, South Korea", network_type="drive")

# 2.1. ROAD_COORD 캐시에 각 노드의 좌표(위도/경도) 저장
ROAD_COORD = {}
for node, data in road_graph.nodes(data=True):
    ROAD_COORD[node] = (data['y'], data['x'])

print("도로망 로드 완료")
print("  - 도로망 노드 수:", road_graph.number_of_nodes())
print("  - 도로망 엣지 수:", road_graph.number_of_edges())

도로망 로드 완료
  - 도로망 노드 수: 68326
  - 도로망 엣지 수: 190081


나중에 사용자 위치도 매핑 시켜야함. walk 그래프에.

In [ ]:
# Step1: 사용자 출발지, 도착지 입력 및 지오코딩

# 1.1 geolocator 초기화
geolocator = Nominatim(user_agent="route_planner")

# 1.2 지오코딩 함수 정의
def get_location(name):
    loc = geolocator.geocode(name, timeout=10)
    if not loc:
        loc = geolocator.geocode(f"{name}, Seoul", timeout=10)
    if not loc:
        raise ValueError(f"'{name}' 위치를 찾을 수 없습니다.")
    return loc.latitude, loc.longitude

# 1.3 사용자에게 출발지와 도착지 명칭 입력받기
start_name = input("출발지 (예: 고려대학교): ")
end_name   = input("도착지 (예: 한남힐스테이트): ")

# 1.4 입력값을 위경도로 변환
start_lat, start_lon = get_location(start_name)
end_lat,   end_lon   = get_location(end_name)

# 1.5 결과 확인용 출력
print(f"출발지 '{start_name}' -> 위도: {start_lat:.6f}, 경도: {start_lon:.6f}")
print(f"도착지 '{end_name}' -> 위도: {end_lat:.6f},   경도: {end_lon:.6f}")

출발지 (예: 고려대학교): 고려대학교
도착지 (예: 한남힐스테이트): 강남역
출발지 '고려대학교' -> 위도: 37.587963, 경도: 127.032346
도착지 '강남역' -> 위도: 37.497541,   경도: 127.026883


Haversine 직선 거리(미터): 10065.997667197036


In [ ]:
# 예시 좌표 (출발/도착 좌표)
start_lat, start_lon = 37.587963, 127.032346
end_lat,   end_lon   = 37.497541, 127.026883

u = ox.distance.nearest_nodes(road_graph, start_lon, start_lat)
v = ox.distance.nearest_nodes(road_graph, end_lon,   end_lat)
print("출발 노드 ID:", u, "도착 노드 ID:", v)


출발 노드 ID: 3828715699 도착 노드 ID: 10220955579


In [ ]:
# 노드가 제대로 선택되었다면 실제 최단 경로 길이를 구해 본다
try:
    length_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    print("도로망 최단경로 길이(미터):", length_m)
except nx.NetworkXNoPath:
    print("두 노드가 같은 연결 성분에 속해 있지 않습니다. (No path)")


도로망 최단경로 길이(미터): 11502.31248291284


In [ ]:
dist = road_distance(start_lat, start_lon, end_lat, end_lon)
print("도로망 기준 거리(미터):", dist)

도로망 기준 거리(미터): 11502.31248291284


In [ ]:
# 지하철 관련 전역 변수 (Step3에서 채울 예정)
subway_info = {}    # { station_name: {'lat': float, 'lng': float, 'lines': set([...])}, ... }
subway_graph = {}   # { (station_name, line): [ (neighbor_station, neighbor_line, weight), ... ], ... }

# 버스 관련 전역 변수 (Step4에서 채울 예정)
bus_info = {}       # { stop_id: {'name': str, 'lat': float, 'lng': float}, ... }
bus_graph = {}      # { stop_id: [ (neighbor_stop_id, weight), ... ], ... }

# 글로벌 속도 및 패널티 설정
AVG_SPEED_KMH_GLOBAL = 18.0   # 대중교통 평균 속도 약 18 km/h (필요시 조정)
WALK_SPEED = 5.0              # 보행 속도 5 km/h
WALK_PENALTY = 3.0            # 환승 시 부과되는 보행+대기 패널티 3분


In [ ]:
# 2. 유틸리티 함수 정의
def haversine(lat1, lon1, lat2, lon2):
    """두 지점 간 대원거리 계산 (미터)"""
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    c = 2 * asin(sqrt(a))
    return 6371000 * c

def road_distance(lat1, lon1, lat2, lon2):
    """도로망 최단 경로 길이(미터) 또는 haversine fallback"""
    try:
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        return nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        return haversine(lat1, lon1, lat2, lon2)

In [ ]:
# 1~9호선 및 기타 전철·경전철 노선별로 직접 지정
speed_by_line = {
    '1호선': 28.6, '2호선': 36.5, '3호선': 34.9, '4호선': 30.1,
    '5호선': 32.8, '6호선': 30.0, '7호선': 32.3, '8호선': 32.2,
    '9호선': 26.2,
    '인천1호선': 32.5, '인천2호선': 32.3,
    '경강선': 68.6, '경의중앙선': 41.5, '경춘선': 60.0,
    '공항철도': 66.7, '서해선': 41.0, '수인분당선': 41.1,
    '신분당선': 54.9, '신림선': 26.5, '우이신설선': 28.7,
    '김포골드라인': 45.0, '용인에버라인': 41.5,
    '의정부경전철': 28.2, 'GTXA': 31.1
}

In [ ]:
# 전역으로 이미 로드되어 있어야 할 것들:
#   road_graph  ──> OSMnx로 만든 도로망 그래프 (networkx.Graph or MultiDiGraph)
#   haversine   ──> (lat1, lon1, lat2, lon2) 간 거리(미터) 계산 함수
#   WALK_SPEED, WALK_PENALTY ──> 보행 속도(km/h), 환승 페널티(분) 등

def build_metro_graph(
    station_json: str,
    line_json: str,
    speed_by_line: dict
):
    """
    station_json: 지하철 역 정보 JSON 파일 경로
    line_json:    지하철 노선 연결 정보 JSON 파일 경로
    speed_by_line: { '1호선': 30.0, '2호선': 32.0, ... }처럼 호선별 평균 속도( km/h ) 딕셔너리

    - station_json의 각 원소 s는 최소한 {'station_cd','name','lat','lng'} 키를 가져야 합니다.
    - line_json의 각 원소 info는 {'line':호선명, 'node':[ {'station':[ {...}, {...} ]}, ... ]} 구조이어야 합니다.
    """
    # 1) JSON 데이터 로드
    stations = json.load(open(station_json, 'r', encoding='utf-8'))['DATA']
    lines    = json.load(open(line_json,    'r', encoding='utf-8'))['DATA']

    # 2) station_cd → {name, lat, lng} 사전 생성
    station_map = {
        s['station_cd']: {
            'name': s.get('name', s.get('station_nm')),
            'lat':  float(s['lat']),
            'lng':  float(s['lng'])
        }
        for s in stations
    }

    # 3) 각 역별 좌표 배열과 KDTree 생성 (후에 “사용자 좌표 → nearest 역 인덱스” 용도로도 사용 가능)
    station_ids    = list(station_map.keys())                    # 예: ['1001','1002', ...]
    coords_metro    = np.array([[station_map[_id]['lat'], station_map[_id]['lng']] for _id in station_ids])
    station_tree    = cKDTree(coords_metro)

    # 4) 도로망 그래프 상의 노드 매핑: station_cd → nearest road_graph 노드 ID
    #    (nearest_nodes(road_graph, lon_list, lat_list)를 쓰면 병렬 매핑 가능)
    lats = [station_map[_id]['lat'] for _id in station_ids]
    lons = [station_map[_id]['lng'] for _id in station_ids]
    road_nodes = nearest_nodes(road_graph, lons, lats)  # road_graph는 OSMnx로 만든 driving 네트워크
    station_to_road = dict(zip(station_ids, road_nodes))
    #    → station_to_road['1001'] = road_graph 내 가장 가까운 노드 ID

    # 5) Graph 초기화 (무향 그래프로 하면 승하차, 상하행 모두 표현 가능)
    Gm = nx.Graph()

    # 6) “역 노드” 추가: node key는 "(station_cd)_(line)" 형태
    #    예: station_cd='1001', line='2호선' → 노드명 = '1001_2호선'
    for idx_cd in station_ids:
        # 역 하나당 여러 호선에 걸쳐 있으면 나중에 환승 엣지로 연결되므로,
        # 우선 모든 (station_cd, line) 조합을 빈 노드로 생성해 두진 않습니다.
        # 실제 엣지를 추가하는 시점에 노드를 add_node 해 줍니다.
        pass

    # 7) 호선별 인접역 엣지 추가
    #    info['node'] 안에 “[{ 'station':[ { ...A역... },{ ...B역... } ] }, ... ]” 구조 가정
    for info in lines:
        line_name = info['line']
        speed_kmh = speed_by_line.get(line_name, 30.0)  # 기본 30 km/h
        for seg in info.get('node', []):
            # seg['station'] = [ {station_cd:A}, {station_cd:B} ]
            a = seg['station'][0]
            b = seg['station'][1]
            u_cd = a['station_cd']
            v_cd = b['station_cd']

            # station_map에서 위/경도 꺼내기
            na = station_map[u_cd]
            nb = station_map[v_cd]

            # 실제 노드명 (station_cd + "_" + line_name)
            u_node = f"{line_name}_{na['name']}"
            v_node = f"{line_name}_{nb['name']}"

            # 노드 속성으로 추가 (node가 없으면 새로 만드는 방식)
            if not Gm.has_node(u_node):
                Gm.add_node(
                    u_node,
                    station = na['name'],
                    line    = line_name,
                    lat     = na['lat'],
                    lng     = na['lng']
                )
            if not Gm.has_node(v_node):
                Gm.add_node(
                    v_node,
                    station = nb['name'],
                    line    = line_name,
                    lat     = nb['lat'],
                    lng     = nb['lng']
                )

            # 7-1) 도로망 기반 거리 계산 (미터)
            #      u_cd → road_nodes[u_cd], v_cd → road_nodes[v_cd]
            u_rg = station_to_road[u_cd]
            v_rg = station_to_road[v_cd]
            try:
                dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
            except (nx.NetworkXNoPath, Exception):
                # 도로망에 경로가 없으면 허버사인 fallback
                dist_m = haversine(na['lat'], na['lng'], nb['lat'], nb['lng'])

            # 7-2) 시간 weight 계산 (분 단위)
            t_min = dist_m / 1000 / speed_kmh * 60  # (km)/ (km/h) * 60

            # 7-3) 실제 엣지 추가
            Gm.add_edge(u_node, v_node, weight=t_min, mode='metro_edge')

    # 8) 동일 역명(같은 station_cd) × 다른 호선 간 환승 엣지 추가
    #    station_map의 key=station_cd가 같으면, dist는 허버사인으로 계산하여
    #    환승 시간 = (거리/보행속도) + 패널티
    from collections import defaultdict
    st2nodes = defaultdict(list)
    for node, data in Gm.nodes(data=True):
        full_name = data['station']  # 예: "고려대(2호선)" vs "고려대(6호선)" 같이 동일 station 컬럼일 수 있음
        st2nodes[full_name].append(node)

    for same_station, nodes_list in st2nodes.items():
        if len(nodes_list) < 2:
            continue
        # 한 역에 같은 이름(station)으로 여러 호선 노드가 있으면 서로 연결
        for i in range(len(nodes_list)):
            for j in range(i+1, len(nodes_list)):
                u_node = nodes_list[i]
                v_node = nodes_list[j]
                lat_u, lng_u = Gm.nodes[u_node]['lat'], Gm.nodes[u_node]['lng']
                lat_v, lng_v = Gm.nodes[v_node]['lat'], Gm.nodes[v_node]['lng']
                walk_m = haversine(lat_u, lng_u, lat_v, lng_v)
                wt = walk_m / 1000 / WALK_SPEED * 60 + WALK_PENALTY
                Gm.add_edge(u_node, v_node, weight=wt, mode='transfer')
                Gm.add_edge(v_node, u_node, weight=wt, mode='transfer')

    return Gm, station_ids, station_tree


In [ ]:
# 올바른 언패킹 예시
metro_graph, station_ids, station_tree = build_metro_graph(
    'data-metro-station-1.0.0.json',
    'data-metro-line-1.0.0.json',
    speed_by_line
)

print(f"지하철 그래프 노드 개수: {metro_graph.number_of_nodes()}")
print(f"지하철 그래프 엣지 개수: {metro_graph.number_of_edges()}")


지하철 그래프 노드 개수: 785
지하철 그래프 엣지 개수: 933


In [ ]:
from collections import defaultdict

In [ ]:
def compute_bus_edge(task):
    """
    task: (u_idx, v_idx, lines_set)
    STOP_COORD, STOP_TO_ROAD, road_graph, AVG_SPEED_KMH_GLOBAL 전역 사용.

    # 버스 구간도 OSM 도로망 기반 최단경로로 계산하도록 변경
    """
    u_idx, v_idx, lines = task

    # 1) 허버사인 대신, ‘정류장 idx → 가장 가까운 도로망 노드’ 매핑 사용
    u_rg = STOP_TO_ROAD[u_idx]
    v_rg = STOP_TO_ROAD[v_idx]

    # 2) OSM 도로망에서 최단 경로 길이(미터) 구하기 (weight='length')
    try:
        dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 도로망 경로가 없으면 허버사인으로 fallback
        y1, x1 = STOP_COORD[u_idx]
        y2, x2 = STOP_COORD[v_idx]
        dist_m = haversine(y1, x1, y2, x2)

    # 3) 거리 → 버스 평균 속도(AVG_SPEED_KMH_GLOBAL)로 분 단위 환산
    t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60
    return u_idx, v_idx, t_min, lines


In [ ]:
 def build_bus_graph_optimized(
    stops_excel: str,
    routes_excel: str,
    avg_speed_kmh: float = 18.0
):
    """
    1) stops_excel: 'bus_stops_seoul.xlsx' 경로 (컬럼: ['정류소명','X좌표','Y좌표'])
    2) routes_excel: 'bus_route.xlsx' 경로 (컬럼: ['노선명','정류소명','순번','X좌표','Y좌표'])
    3) avg_speed_kmh: 버스 평균 속도(km/h), 기본 18.0

    - 버스 정류장 간 좌표 매핑 → 잘못 매핑된(거리 차이 큰) 로우들은 자동으로 드롭
    - STOP_TO_ROAD: 각 정류장 인덱스를 OSM 도로망 노드 ID로 매핑하고,
      동일 도로망 노드에 매핑된 정류장이 몇 개 있는지(overlap)를 출력
    - 노드명 = f"{노선명}_{정류소명}"
    - 버스 이동 엣지 생성 (OSM 도로망 최단경로 이용)
    - 보행 환승 엣지 생성 (1,000m 이내, 20분 이내 걸어갈 수 있으면)
    - 반환값: (bus_graph, edge_lines)
      • bus_graph: networkx.Graph (nodes: 노선명_정류소명, edges: mode, weight)
      • edge_lines: { (u_node, v_node): set(노선명) }
    """

    global STOP_COORD, STOP_TO_ROAD, AVG_SPEED_KMH_GLOBAL
    AVG_SPEED_KMH_GLOBAL = avg_speed_kmh

    # ─────────────────────────────────────────────────────────────────────────
    # 1) 정류장 엑셀 불러오기 & KDTree 생성
    stops_df = pd.read_excel(stops_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols = {'정류소명','X좌표','Y좌표'}
    if not required_cols.issubset(stops_df.columns):
        raise ValueError(f"stops_excel에 반드시 {required_cols} 컬럼이 있어야 합니다.")

    # 인덱스가 정류장 고유 ID 역할 (0, 1, 2, …)
    stops_df = stops_df.reset_index(drop=True)

    # (1-1) KDTree 생성: stops_df의 (위도, 경도) 배열
    coords_bus = stops_df[['Y좌표','X좌표']].to_numpy()  # shape=(N,2)
    stop_tree  = cKDTree(coords_bus)

    # ─────────────────────────────────────────────────────────────────────────
    # 2) 노선 정보 엑셀 불러오기 & 좌표 기반 매핑 → 잘못된 매핑(drop)
    routes_df = pd.read_excel(routes_excel, dtype={'X좌표': float, 'Y좌표': float})
    required_cols2 = {'노선명','정류소명','순번','X좌표','Y좌표'}
    if not required_cols2.issubset(routes_df.columns):
        raise ValueError(f"routes_excel에 반드시 {required_cols2} 컬럼이 있어야 합니다.")

    # (2-1) routes_df 각 행의 (위도,경도)를 KDTree 로 쿼리 → nearest stops_df 인덱스 찾기
    route_coords = routes_df[['Y좌표','X좌표']].to_numpy()
    dists, nearest_idx = stop_tree.query(route_coords, k=1)
    routes_df['matched_stop_idx'] = nearest_idx.astype(int)
    routes_df['matched_dist_m'] = [
        haversine(lat1, lon1,
                  stops_df.loc[idx,'Y좌표'], stops_df.loc[idx,'X좌표'])
        for (lat1, lon1), idx in zip(route_coords, nearest_idx)
    ]

    # (2-2) 특정 거리 임계치 이상(여기서는 50m) 차이가 나는 경우 “잘못된 좌표”로 보고 드롭
    DROP_THRESHOLD_M = 50.0
    mask_good = routes_df['matched_dist_m'] <= DROP_THRESHOLD_M
    num_dropped = (~mask_good).sum()
    if num_dropped > 0:
        print(f">>> **{num_dropped}개의 노선-정류장 로우는 KDTree 매핑 거리가 {DROP_THRESHOLD_M}m 초과하여 드롭됩니다.**")
    routes_df = routes_df[mask_good].copy()

    # ─────────────────────────────────────────────────────────────────────────
    # 3) 버스 그래프 초기화 & 노드 생성 준비
    bus_graph = nx.Graph()
    # edge_lines: “(u_node, v_node) → {노선명 set}”
    edge_lines = defaultdict(set)

    # STOP_COORD: 정류장 idx → (lat, lon)
    STOP_COORD = {
        idx: (row['Y좌표'], row['X좌표'])
        for idx, row in stops_df.iterrows()
    }

    # ─────────────────────────────────────────────────────────────────────────
    # 4) STOP_TO_ROAD (정류장 idx → OSM 도로망 노드 ID) 매핑
    #    (lon, lat) 순서로 nearest_nodes 호출
    lats = stops_df['Y좌표'].values
    lons = stops_df['X좌표'].values
    road_nodes = ox.distance.nearest_nodes(road_graph, lons, lats)
    STOP_TO_ROAD = dict(enumerate(road_nodes))

    # 4-1) 동일한 OSM 노드에 매핑된 정류장(들) 찾기 → overlap 체크
    inv = defaultdict(list)
    for stop_idx, rnode in STOP_TO_ROAD.items():
        inv[rnode].append(stop_idx)

    overlap_count = 0
    overlap_groups = []
    for rnode, idx_list in inv.items():
        if len(idx_list) > 1:
            overlap_groups.append(idx_list)
            overlap_count += len(idx_list)
    if overlap_count > 0:
        print(f">>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 {overlap_count}개**")
        print("   - 예시 겹침 그룹 (stop_idx 리스트):")
        for grp in overlap_groups:
            print(f"     → {grp}")

    # ─────────────────────────────────────────────────────────────────────────
    # 5) “노선명_정류소명” 노드 & 버스 엣지 생성
    #    → 각 노선별로 ‘순번’ 순으로 정차 정류장 인덱스를 정렬한 뒤 인접 쌍(u_idx, v_idx)에 대해 엣지 추가
    for line_name, grp in routes_df.groupby('노선명'):
        grp_sorted = grp.sort_values('순번')
        prev_idx = None
        for _, row in grp_sorted.iterrows():
            cur_idx = int(row['matched_stop_idx'])
            stop_name = stops_df.loc[cur_idx, '정류소명']

            # 노드명: "노선명_정류소명"
            this_node = f"{line_name}_{stop_name}"

            # 실제 노드가 없으면 생성 (노드 속성: coord, stop_idx, 노선명)
            if not bus_graph.has_node(this_node):
                lat0, lon0 = STOP_COORD[cur_idx]
                bus_graph.add_node(
                    this_node,
                    coord     = (lat0, lon0),
                    stop_idx  = cur_idx,
                    route     = line_name
                )

            if prev_idx is not None and prev_idx != cur_idx:
                prev_stop_name = stops_df.loc[prev_idx, '정류소명']
                prev_node = f"{line_name}_{prev_stop_name}"

                # 5-1) 버스 구간 거리 계산 (OSM 도로망 경로 길이)
                u_rg = STOP_TO_ROAD[prev_idx]
                v_rg = STOP_TO_ROAD[cur_idx]
                try:
                    dist_m = nx.shortest_path_length(road_graph, u_rg, v_rg, weight='length')
                except (nx.NetworkXNoPath, Exception):
                    # 도로망 연결 없으면 직선거리 fallback
                    lat_u, lon_u = STOP_COORD[prev_idx]
                    lat_v, lon_v = STOP_COORD[cur_idx]
                    dist_m = haversine(lat_u, lon_u, lat_v, lon_v)

                # 5-2) 소요 시간 계산(분) = 거리(m) / 1000 / 속도(km/h) * 60
                t_min = dist_m / 1000 / AVG_SPEED_KMH_GLOBAL * 60

                # 5-3) 양방향 버스 엣지 추가
                bus_graph.add_edge(
                    prev_node, this_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )
                bus_graph.add_edge(
                    this_node, prev_node,
                    weight = t_min,
                    mode   = 'bus_edge'
                )

                # 5-4) edge_lines에 해당 엣지를 어떤 노선이 쓰는지 기록
                edge_lines[(prev_node, this_node)].add(line_name)
                edge_lines[(this_node, prev_node)].add(line_name)

            prev_idx = cur_idx

    # ─────────────────────────────────────────────────────────────────────────
    # 6) 보행 환승 엣지 추가 (1,000m 반경 / 20분 이내)
    TRANSFER_RADIUS_M = 1000.0
    # 반경(deg) 단순 변환 (위도 1° ≈ 111 km) → 대략적인 검색용
    radius_deg = TRANSFER_RADIUS_M / 111000.0

    # stop_tree는 1번에서 생성한 KDTree
    # coords_bus: (위도,경도) 배열
    for u_idx in range(len(stops_df)):
        # 이 정류장(u_idx) 반경 radius_deg 내 이웃 정류장 인덱스 리스트
        nbrs = stop_tree.query_ball_point(coords_bus[u_idx], r=radius_deg)
        for v_idx in nbrs:
            if v_idx <= u_idx:
                continue

            # 실제 직선 거리(미터)
            lat_u, lon_u = STOP_COORD[u_idx]
            lat_v, lon_v = STOP_COORD[v_idx]
            dist_uv = haversine(lat_u, lon_u, lat_v, lon_v)

            # “걸어서 걸리는 시간(분)”
            walk_time = dist_uv / 1000 / WALK_SPEED * 60

            if walk_time <= 20.0:  # 20분 이내(≒1km)라면 환승 가능
                # u_idx, v_idx에 해당하는 모든 “노선별 노드”를 서로 연결
                # (stop_idx → 어떤 route들이 쓰고 있는지 알아야 함)
                # 따라서 먼저 “이 정류장 u_idx를 쓰는 노드들”을 찾자
                u_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == u_idx]
                v_nodes = [node for node, data in bus_graph.nodes(data=True)
                           if data.get('stop_idx') == v_idx]

                for un in u_nodes:
                    for vn in v_nodes:
                        # 이미 버스 엣지가 있으면 skip (버스 엣지가 있으면 mode='bus_edge'이므로)
                        if bus_graph.has_edge(un, vn):
                            continue
                        # u→v 환승 엣지
                        bus_graph.add_edge(
                            un, vn,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )
                        # v→u 환승 엣지
                        bus_graph.add_edge(
                            vn, un,
                            weight = walk_time + WALK_PENALTY,
                            mode   = 'transfer'
                        )

    return bus_graph, edge_lines


In [ ]:
# 4.3. build_bus_graph_optimized 함수 실행 및 결과 확인
bus_graph, edge_lines = build_bus_graph_optimized('bus_stops_seoul.xlsx', 'bus_route.xlsx', avg_speed_kmh=18.0)

print(f"최종 버스 그래프 노드(정류장) 개수: {bus_graph.number_of_nodes()}")
print(f"최종 버스 그래프 엣지(유니크) 개수: {bus_graph.number_of_edges()}")

>>> **5469개의 노선-정류장 로우는 KDTree 매핑 거리가 50.0m 초과하여 드롭됩니다.**
>>> **OSM 노드 상에서 겹치는(같은 노드에 매핑된) 정류장 인덱스 총 6453개**
   - 예시 겹침 그룹 (stop_idx 리스트):
     → [1, 117, 232, 366, 369, 371, 376]
     → [8, 45]
     → [12, 262]
     → [20, 268]
     → [26, 28]
     → [27, 37]
     → [29, 34]
     → [30, 33]
     → [31, 32]
     → [35, 199, 271, 381]
     → [38, 51]
     → [39, 50, 289, 290]
     → [40, 264]
     → [43, 143, 380, 382]
     → [48, 281]
     → [54, 99, 378]
     → [55, 68, 363]
     → [59, 64]
     → [65, 148]
     → [69, 75]
     → [70, 132]
     → [71, 72]
     → [73, 362]
     → [78, 83]
     → [85, 167]
     → [87, 211]
     → [88, 5110]
     → [89, 543]
     → [90, 5092]
     → [91, 98, 293]
     → [100, 101, 142, 276, 372]
     → [102, 373]
     → [111, 116]
     → [112, 144, 302]
     → [113, 114, 375]
     → [115, 227, 365, 370]
     → [120, 249]
     → [121, 243, 364]
     → [145, 296]
     → [147, 253]
     → [151, 2030]
     → [155, 156, 291, 292]
     → [160, 168, 311, 315]
 

In [ ]:
# 예시: "bus_2840" 와 "bus_9421" 의 정류장 인덱스(idx) 가 각각 무엇인지 알아낸 뒤 그 값을 넣어서 테스트

# 1) 정류장 idx → road node ID 출력
print("=== STOP_TO_ROAD 샘플 매핑 확인 ===")
sample_indices = [  # 여기에는 실제 버스 노드 인덱스를 넣어주세요. 예제에서는 2840, 9421 가 아니라
                   # stops_df 에서 할당된 idx(0~n-1) 값이어야 합니다.
                   # 코드 전체에서 가장 먼저 'bus_2840' 처럼 쓰인 건 "bus_graph 노드 키"이지,
                   # STOP_TO_ROAD에서는 정수 idx 그대로를 사용했으니,
                   # stops_df 내부에서 '정류장ID'가 몇 번째 인덱스인지 찾아서 써야 합니다.
                   2840, 9421
                 ]
for idx in sample_indices:
    if idx in STOP_TO_ROAD:
        print(f"정류장 idx {idx} → road_node ID: {STOP_TO_ROAD[idx]}")
    else:
        print(f"정류장 idx {idx} 은 STOP_TO_ROAD에 없습니다.")

# 2) 만약 동일 ID가 찍히면, 두 정류장은 사실상 road_graph 상 같은 노드를 가리키고 있는 것
if len(sample_indices) == 2:
    u_rg = STOP_TO_ROAD.get(sample_indices[0])
    v_rg = STOP_TO_ROAD.get(sample_indices[1])
    print("두 정류장 road-node 동일 여부:", u_rg == v_rg)


=== STOP_TO_ROAD 샘플 매핑 확인 ===
정류장 idx 2840 → road_node ID: 7579130762
정류장 idx 9421 → road_node ID: 10220955579
두 정류장 road-node 동일 여부: False


In [ ]:
# --- 5.0. 사전 준비: 필요한 전역 변수 및 함수가 이미 정의되었다고 가정 ---
# - metro_graph: Step3에서 생성된 지하철 그래프
# - bus_graph:   Step4에서 생성된 버스 그래프
# - STOP_COORD:  { idx: (lat, lon) } (버스 정류장 인덱스 → 위/경도)
# - WALK_SPEED, WALK_PENALTY: 보행 속도(km/h), 환승 패널티(분)
# - haversine(lat1, lon1, lat2, lon2): 두 지점 간 거리(미터) 계산 함수

# 5.1. 통합 그래프 초기화
integrated_graph = nx.Graph()


In [ ]:
# ─── 1. 지하철 노드·엣지 복사 ─────────────────────────────────────────
for node, data in metro_graph.nodes(data=True):
    integrated_graph.add_node(
        node,
        mode='metro',
        station=data.get('station'),
        line=data.get('line'),
        lat=data.get('lat'),
        lng=data.get('lng')
    )

for u, v, data in metro_graph.edges(data=True):
    w = data.get('weight')
    if data.get('mode') == 'transfer':
        integrated_graph.add_edge(u, v, weight=w, mode='transfer')
    else:
        integrated_graph.add_edge(u, v, weight=w, mode='metro_edge')


In [ ]:
# ─── 2. 버스 노드·엣지 복사 ─────────────────────────────────────────
for idx, data in bus_graph.nodes(data=True):
    bus_node = f"bus_{idx}"
    lat0, lon0 = data['coord']
    integrated_graph.add_node(
        bus_node,
        mode='bus',
        stop_idx=idx,
        name=data.get('name'),
        lat=lat0,
        lng=lon0
    )

for u, v, data in bus_graph.edges(data=True):
    u_key = f"bus_{u}"
    v_key = f"bus_{v}"
    integrated_graph.add_edge(
        u_key, v_key,
        weight = data['weight'],
        mode   = data.get('mode', 'bus_edge'),
        lines  = data.get('lines', set())
    )


In [ ]:
# ─── 3. 버스↔지하철 환승 엣지 추가 (1 000m 반경) ────────────────────────

# 3-1) 버스 정류장 좌표 배열 + KDTree 준비
# STOP_COORD: { stop_idx: (lat, lon) } 형태
bus_indices   = list(STOP_COORD.keys())
bus_coords    = np.array([STOP_COORD[s] for s in bus_indices])  # shape=(N_bus, 2) → [(lat,lon), ...]
bus_tree      = cKDTree(bus_coords)

# (위도 1° ≈ 111 km) 정도이므로, 1 000 m 반경을 위/경도 단위로 _대략_ 변환
TRANSFER_RADIUS_M = 1000.0
radius_deg = TRANSFER_RADIUS_M / 111000.0  # ≈ 0.009009°

# 3-2) 각 메트로 노드마다 주변 버스 정류장 후보 찾고, 실제 거리 재확인
for m_node, m_data in metro_graph.nodes(data=True):
    metro_lat, metro_lon = m_data['lat'], m_data['lng']
    # KDTree 에서는 (lat, lon) 순서로 쿼리
    # 반경(radius_deg) 내 후보 버스 정류장 인덱스 배열(idx_in_bus_coords_list)을 돌려준다
    idxs_in_bus_array = bus_tree.query_ball_point([metro_lat, metro_lon], r=radius_deg)

    # 실제 환승 가능한 버스 정류장만 골라서 엣지 추가
    for arr_i in idxs_in_bus_array:
        stop_idx = bus_indices[arr_i]           # STOP_COORD의 실제 stop_idx
        bus_lat, bus_lon = STOP_COORD[stop_idx]

        # 실제 거리를 haversine으로 계산
        dist_m = haversine(bus_lat, bus_lon, metro_lat, metro_lon)
        if dist_m <= TRANSFER_RADIUS_M:
            walk_time     = (dist_m / 1000.0) / WALK_SPEED * 60.0
            transfer_time = walk_time + WALK_PENALTY

            bus_node_key  = f"bus_{stop_idx}"
            integrated_graph.add_edge(
                bus_node_key, m_node,
                weight = transfer_time,
                mode   = 'transfer'
            )


In [ ]:
# ─── 4. 결과 확인 ─────────────────────────────────────────────────
node_count     = integrated_graph.number_of_nodes()
edge_count     = integrated_graph.number_of_edges()
transfer_edges = [(u, v) for u, v, d in integrated_graph.edges(data=True) if d['mode'] == 'transfer']

print("통합 그래프 노드 개수:", node_count)
print("통합 그래프 엣지 개수:", edge_count)
print("버스↔지하철 환승 엣지 개수:", len(transfer_edges))


통합 그래프 노드 개수: 40547
통합 그래프 엣지 개수: 2839895
버스↔지하철 환승 엣지 개수: 2805250


In [37]:
# 통합 그래프에서 버스/지하철 환승 엣지만 샘플 확인
count = 0
for u, v, data in integrated_graph.edges(data=True):
    if data.get('mode') == 'transfer':
        print(f"환승 엣지: {u} ↔ {v}, time={data['weight']:.2f}분")
        count += 1
        if count >= 10:
            break

환승 엣지: 1호선_회룡 ↔ 의정부경전철_회룡, time=3.84분
환승 엣지: 1호선_도봉산 ↔ 7호선_도봉산, time=3.62분
환승 엣지: 1호선_도봉산 ↔ bus_3640, time=9.45분
환승 엣지: 1호선_도봉산 ↔ bus_3502, time=6.55분
환승 엣지: 1호선_도봉산 ↔ bus_3501, time=9.51분
환승 엣지: 1호선_도봉산 ↔ bus_3653, time=5.07분
환승 엣지: 1호선_도봉산 ↔ bus_3651, time=4.70분
환승 엣지: 1호선_도봉산 ↔ bus_3652, time=4.87분
환승 엣지: 1호선_도봉산 ↔ bus_3503, time=6.46분
환승 엣지: 1호선_도봉산 ↔ bus_3457, time=12.91분


############

In [ ]:
# (Step 5가 끝난 뒤) --- 6.1. nearby_integrated_nodes 함수 정의 ---
def nearby_integrated_nodes(lat, lon, radius_m=500):
    result = []
    for node, data in integrated_graph.nodes(data=True):
        node_lat = data.get('lat')
        node_lon = data.get('lng')
        if node_lat is None or node_lon is None:
            continue
        dist_m = haversine(lat, lon, node_lat, node_lon)
        if dist_m <= radius_m:
            result.append((node, dist_m))
    return result


In [ ]:
def walk_time_via_road(lat1, lon1, lat2, lon2):
    """
    두 지점(위도·경도) 사이를 도로망(graph=road_graph) 상에서 최단 경로를 따라
    이동할 때 걸리는 시간(분)을 계산합니다.
    도로망에 경로가 없다면 허버사인 거리로 fallback합니다.
    """
    try:
        # 1) 가까운 도로망 노드로 매핑
        u = nearest_nodes(road_graph, lon1, lat1)
        v = nearest_nodes(road_graph, lon2, lat2)
        # 2) 도로망 최단 경로 길이(미터)
        dist_m = nx.shortest_path_length(road_graph, u, v, weight='length')
    except (nx.NetworkXNoPath, Exception):
        # 도로망 경로가 없으면 허버사인 거리로 fallback
        dist_m = haversine(lat1, lon1, lat2, lon2)
    # 3) 보행 속도 WALK_SPEED(=5 km/h) 로 분 단위 환산
    return (dist_m / 1000) / WALK_SPEED * 60


In [ ]:
# 1) 후보 노드 생성 & 거리순 정렬
start_candidates = nearby_integrated_nodes(start_lat, start_lon, radius_m=500)
end_candidates   = nearby_integrated_nodes(end_lat,   end_lon,   radius_m=500)

start_candidates.sort(key=lambda x: x[1])  # (node, dist_m) 리스트 → dist_m 순 정렬
end_candidates.sort(key=lambda x: x[1])

# 3) 도보 시간 계산 (도로망 기반)
start_nodes_with_walk = []
for node, _ in start_candidates:
    node_lat = integrated_graph.nodes[node]['lat']
    node_lon = integrated_graph.nodes[node]['lng']
    walk_min = walk_time_via_road(start_lat, start_lon, node_lat, node_lon)
    start_nodes_with_walk.append((node, walk_min))

end_nodes_with_walk = []
for node, _ in end_candidates:
    node_lat = integrated_graph.nodes[node]['lat']
    node_lon = integrated_graph.nodes[node]['lng']
    walk_min = walk_time_via_road(node_lat, node_lon, end_lat, end_lon)
    end_nodes_with_walk.append((node, walk_min))


In [38]:
def get_fastest_routes_optimized(G, origin_candidates, dest_candidates, k=10, max_transfers=4):
    """
    - origin_candidates: [(origin_node, origin_walk_time), …]
    - dest_candidates:   [(dest_node,   dest_walk_time),   …]
    - k: 반환할 상위 개수
    - max_transfers: 허용할 최대 환승 횟수
    """
    fastest_list = []

    # 1) origin 후보 하나당 single_source_dijkstra 한 번 실행
    for origin_node, origin_walk in origin_candidates:
        # (a) origin_node → 전 노드 최단 경로 한 번에 계산
        distances, paths = nx.single_source_dijkstra(
            G,
            origin_node,
            weight='weight'
        )
        # distances: { node: 최단가중치합(시간) }
        # paths:     { node: [origin_node → … → node] } (노드 리스트)

        # 2) 모든 dest 후보를 한꺼번에 조회
        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue  # 경로가 없는 경우 건너뛰기

            travel_time = distances[dest_node]  # origin_node→dest_node 내부 교통시간
            path = paths[dest_node]             # origin_node→dest_node 실제 경로 리스트

            total_time = travel_time + origin_walk + dest_walk

            # 3) 경로(path)를 순회해 환승 횟수 계산
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                ed = G[u][v]
                mode = ed.get('mode')
                if mode == 'transfer':
                    if 0 < idx < (len(edges)-1):
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if mode == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if mode == 'bus_edge':
                    cur_br = next(iter(ed.get('lines', [])), None)
                    if prev_mode == 'bus_edge' and prev_line != cur_br:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_br
                    continue
                prev_mode, prev_line = mode, None

            # 4) 환승 제한 검사
            if transfer_count > max_transfers:
                continue

            total_walk = origin_walk + dest_walk
            fastest_list.append({
                'path': path,
                'time_min': total_time,
                'transfers': transfer_count,
                'walk_min': total_walk
            })

    # 5) 최종 정렬 후 상위 k개 반환
    fastest_list.sort(key=lambda x: x['time_min'])
    return fastest_list[:k]


In [39]:
def get_least_transfer_routes_optimized(G, origin_candidates, dest_candidates, k=10, max_transfers=4):
    """
    - transfer_weight: 엣지별 “환승시 10000 패널티 + 통행시간” 으로 가중치 계산
    """
    def transfer_weight(u, v, edge_data):
        mode = edge_data.get('mode')
        w = edge_data.get('weight', 0)
        if mode == 'transfer':
            return 10000 + w
        if mode == 'metro_edge':
            lu = G.nodes[u].get('line')
            lv = G.nodes[v].get('line')
            if lu is None or lv is None or lu != lv:
                return 10000 + w
            return w
        if mode == 'bus_edge':
            return w
        return w

    least_transfer_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) origin_node 한 번에 Dijkstra → distances, paths 얻기
        distances, paths = nx.single_source_dijkstra(
            G,
            origin_node,
            weight=transfer_weight
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            # (a) distances[dest_node]에는 “환승 패널티 포함 + 내부 통행시간”이 누적된 값
            # → 실제 여행 시간(내부 교통시간)만 따로 계산하려면 경로를 순회해야 함.
            path = paths[dest_node]

            # 2) 실제 환승 횟수 계산 (기존과 동일)
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                ed = G[u][v]
                mode = ed.get('mode')
                if mode == 'transfer':
                    if idx != 0 and idx != (len(edges) - 1):
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if mode == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if mode == 'bus_edge':
                    cur_br = next(iter(ed.get('lines', [])), None)
                    if prev_mode == 'bus_edge' and prev_line != cur_br:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_br
                    continue
                prev_mode, prev_line = mode, None

            if transfer_count > max_transfers:
                continue

            # 3) 실제 내부 통행시간(버스/지하철/환승 weight)은 경로를 순회하며 직접 합산
            travel_time = sum(G[u][v]['weight'] for u, v in edges)

            total_time = travel_time + origin_walk + dest_walk
            total_walk = origin_walk + dest_walk

            least_transfer_list.append({
                'path': path,
                'time_min': total_time,
                'transfers': transfer_count,
                'walk_min': total_walk
            })

    # 4) (환승 횟수, 총 시간) 기준으로 정렬
    least_transfer_list.sort(key=lambda x: (x['transfers'], x['time_min']))
    return least_transfer_list[:k]


In [40]:
def get_least_walking_routes_optimized(G, origin_candidates, dest_candidates, k=10, max_transfers=4):
    def walk_only_weight(u, v, edge_data):
        return edge_data['weight'] if edge_data.get('mode') == 'transfer' else 0

    least_walking_list = []

    for origin_node, origin_walk in origin_candidates:
        # 1) origin_node 한 번에 Dijkstra → distances, paths
        distances, paths = nx.single_source_dijkstra(
            G,
            origin_node,
            weight=walk_only_weight
        )

        for dest_node, dest_walk in dest_candidates:
            if dest_node not in distances:
                continue

            # (a) distances[dest_node] = “경로 중 transfer weight 합”
            walk_length_along_graph = distances[dest_node]
            path = paths[dest_node]

            # 2) 환승 횟수 계산 (기존 로직과 동일)
            transfer_count = 0
            prev_mode, prev_line = None, None
            edges = list(zip(path, path[1:]))

            for idx, (u, v) in enumerate(edges):
                ed = G[u][v]
                mode = ed.get('mode')
                if mode == 'transfer':
                    if 0 < idx < (len(edges)-1):
                        transfer_count += 1
                    prev_mode, prev_line = 'transfer', None
                    continue
                if mode == 'metro_edge':
                    cur_line = G.nodes[u].get('line')
                    if prev_mode == 'metro_edge' and prev_line != cur_line:
                        transfer_count += 1
                    prev_mode, prev_line = 'metro_edge', cur_line
                    continue
                if mode == 'bus_edge':
                    cur_br = next(iter(ed.get('lines', [])), None)
                    if prev_mode == 'bus_edge' and prev_line != cur_br:
                        transfer_count += 1
                    prev_mode, prev_line = 'bus_edge', cur_br
                    continue
                prev_mode, prev_line = mode, None

            if transfer_count > max_transfers:
                continue

            total_walk = walk_length_along_graph + origin_walk + dest_walk
            travel_time = sum(G[u][v]['weight'] for u, v in edges)
            total_time = travel_time + origin_walk + dest_walk

            least_walking_list.append({
                'path': path,
                'time_min': total_time,
                'transfers': transfer_count,
                'walk_min': total_walk
            })

    least_walking_list.sort(key=lambda x: (x['walk_min'], x['time_min']))
    return least_walking_list[:k]


In [41]:
def get_top_routes_optimized(G, origin_candidates, dest_candidates, k_per_criteria=10, max_transfers=4):
    return {
        'fastest': get_fastest_routes_optimized(G, origin_candidates, dest_candidates,
                                                k=k_per_criteria, max_transfers=max_transfers),
        'least_transfers': get_least_transfer_routes_optimized(G, origin_candidates, dest_candidates,
                                                                k=k_per_criteria, max_transfers=max_transfers),
        'least_walking': get_least_walking_routes_optimized(G, origin_candidates, dest_candidates,
                                                            k=k_per_criteria, max_transfers=max_transfers)
    }


시간& 내리는 역 이름

In [42]:
# 4) 실제 호출 예시:
results = get_top_routes_optimized(
    integrated_graph,
    start_nodes_with_walk,   # 반경 500m 이내 후보 + 도로망 기반 도보 시간
    end_nodes_with_walk,
    k_per_criteria=10,       # 각 기준별 상위 10개
    max_transfers=4
)

# 5) 결과 출력
for criterion, routes in results.items():
    print(f"=== {criterion.upper()} ROUTES (max_transfers=4) ===")
    for idx, route in enumerate(routes, 1):
        path_str = " → ".join(route['path'])
        print(f"[{idx}] 경로: {path_str}")
        print(f"     총시간: {route['time_min']:.1f}분 | 환승: {route['transfers']}회 | 도보: {route['walk_min']:.1f}분")
    print()


=== FASTEST ROUTES (max_transfers=4) ===
[1] 경로: 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
     총시간: 42.2분 | 환승: 2회 | 도보: 11.6분
[2] 경로: 6호선_고려대 → 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남
     총시간: 44.6분 | 환승: 2회 | 도보: 12.1분
[3] 경로: 6호선_안암 → 6호선_보문 → 6호선_창신 → 6호선_동묘앞 → 6호선_신당 → 6호선_청구 → 6호선_약수 → 3호선_약수 → 3호선_금호 → 3호선_옥수 → 3호선_압구정 → 3호선_신사 → 신분당선_신사 → 신분당선_논현 → 신분당선_신논현 → 신분당선_강남 → 2호선_강남
     총시간: 47.8분 | 환승: 2회 | 도보: 12.6분
[4] 경로: bus_141_고려대역.고대앞삼거리 → bus_141_제기동한신아파트앞 → bus_141_홍파초등학교 → bus_141_경동시장앞 → bus_141_동대문구청.용신동주민센터 → bus_141_마장축산물시장 → bus_141_도선사거리 → bus_141_성동구청 → bus_141_무학여고앞 → bus_141_응봉사거리 → bus_141_뚝섬서울숲 → bus_141_성수대교남단.현대아파트 → bus_141_신구중학교 → bus_141_도산공원 → bus_141_서울세관 → bus_141_임피리얼팰리스호텔.펜트힐루논현 → bus_141_논현아이파크 → bus_141_시티프라디움더강남.라움아트센터 → bus_141_KT강남지사